<a href="https://colab.research.google.com/github/ella00100/Kaggle_competition/blob/main/Home_Credit_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Introduction to Manual Feature Engineering

- 이 노트북에서는 홈 크레딧 기본 위험 경쟁을 위한 기능을 직접 만드는 방법에 대해 알아봅니다. 이전 노트북에서는 모델을 구축하기 위해 애플리케이션 데이터만 사용했습니다. 우리가 이 데이터에서 만든 최고의 모델은 리더보드에서 약 0.74점을 얻었습니다. 이 점수를 개선하려면 다른 데이터 프레임의 정보를 더 많이 포함해야 합니다. 

- 여기서는 Bureau의 정보와 Bureau_balance 데이터를 사용하는 방법을 살펴보겠습니다. 이러한 데이터 파일의 정의는 다음과 같습니다:

- Bureau: 고객이 Home Credit에 보고한 다른 금융 기관과의 이전 대출에 대한 정보. 이전 대출에는 각각의 행이 있습니다.

- bureau_balance: 이전 대출에 대한 월별 정보. 각 달에는 고유한 행이 있습니다.
manual feature engineering은 지루한 프로세스가 될 수 있으며(기능 툴과 함께 자동화된 기능 엔지니어링을 사용하는 이유입니다!) 종종 도메인 전문 지식에 의존합니다. 대출에 대한 도메인 지식과 채무 불이행 가능성을 높이는 요소가 제한되어 있기 때문에, 대신 최종 교육 데이터 프레임에 가능한 한 많은 정보를 얻는 데 집중할 것입니다. 그 아이디어는 모델이 우리가 결정해야 하는 것보다 어떤 기능이 중요한지 알게 될 것이라는 것입니다. 기본적으로, 우리의 접근 방식은 가능한 한 많은 기능을 만든 다음 이 모든 기능을 모델이 사용하도록 제공하는 것입니다! 나중에 모델의 기능 중요도나 PCA와 같은 다른 기술을 사용하여 기능 축소를 수행할 수 있습니다.

- manual feature engineering 프로세스에는 많은 Pandas 코드, 약간의 인내심 및 많은 우수한 연습 조작 데이터가 포함됩니다. 자동화된 기능 엔지니어링 도구를 사용할 수 있게 되기 시작했지만, 기능 엔지니어링은 여전히 많은 데이터 논쟁을 사용하여 잠시 동안 수행해야 합니다.

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

plt.style.use('fivethirtyeight')

## Ex: Counts of a client's previous loans

- manual feature engineering의 일반적인 프로세스를 설명하기 위해 먼저 다른 금융 기관에서 고객의 이전 대출 금액을 간단히 계산합니다. 이를 위해서는 노트북 전체에 걸쳐 많은 Pandas 작업이 필요합니다:

  - groupby: 데이터 프레임을 열별로 그룹화합니다. 이 경우 고유 고객인 SK_ID_CURR 열별로 그룹화합니다

  - agg: 열의 평균을 구하는 것과 같은 그룹화된 데이터에 대해 계산을 수행합니다. 함수(grouped_df.mean())를 직접 호출하거나 변환 목록(grouped_df.agg([mean, max, min, sum])과 함께 agg 함수를 사용할 수 있습니다

  - merge: 집계된 통계를 적절한 클라이언트와 일치시킵니다. 원래 교육 데이터를 SK의 계산된 통계와 병합해야 합니다클라이언트에 해당 통계가 없는 셀에 NaN을 삽입할 ID_CURR 열
또한 사전으로 이름을 바꿀 열을 지정하는 (이름 바꾸기) 기능을 상당히 많이 사용합니다. 이것은 우리가 생성하는 새로운 변수를 추적하는 데 유용합니다.

- 이것은 많은 것처럼 보일 수 있습니다. 그래서 우리는 결국 이 과정을 수행하기 위한 함수를 작성할 것입니다. 먼저 수작업으로 구현하는 방법을 살펴보겠습니다.

In [ ]:
from google.colab import files
files.upload()
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c home-credit-default-risk
!ls

In [ ]:
!unzip home-credit-default-risk.zip

In [ ]:
bureau = pd.read_csv('bureau.csv')
bureau.head()

In [ ]:
# bureau 데이터프레임을 'SK_ID_CURR'로 그룹화하고, 'SK_ID_BUREAU'열을 count하여 'previous_loan_counts'라는 새로운 열을 생성
previous_loan_counts = bureau.groupby(
    'SK_ID_CURR', as_index=False)['SK_ID_BUREAU'].count().rename(columns = {'SK_ID_BUREAU': 'previous_loan_counts'})

previous_loan_counts.head()

In [ ]:
train = pd.read_csv('application_train.csv')

#이전 대출 횟수 정보를 'train' 데이터프레임과 합침침
train = train.merge(previous_loan_counts, on = 'SK_ID_CURR', how='left')

#'previous_loan_counts' 열에서 결측값이 있으면 0
train['previous_loan_counts'] = train['previous_loan_counts'].fillna(0)
train.head()

### r값을 이용한 새로운 변수의 유용성 평가
- 새 변수가 유용한지 여부를 확인하기 위해 이 변수와 목표값 사이의 피어슨 상관 계수(r-값)를 계산할 수 있습니다. 이 값은 두 변수 사이의 선형 관계의 강도를 측정하며 -1(완전히 음의 선형)에서 +1(완전히 양의 선형)까지의 범위를 갖습니다. 

- r-값은 새 변수의 "유용성"에 대한 최상의 척도는 아니지만 변수가 기계 학습 모델에 도움이 되는지 여부에 대한 첫 번째 근사치를 제공할 수 있습니다. 목표값에 대한 변수의 r-값이 클수록 이 변수의 변화는 목표값에 더 많은 영향을 미칠 수 있습니다. 따라서 목표값과 관련하여 절대값 r-값이 가장 큰 변수를 찾습니다.

- 또한 커널 밀도 추정(KDE) 그림을 사용하여 대상과의 관계를 시각적으로 검사할 수 있습니다.

#### Kernel Density Estimate Plots

- 커널 밀도 추정치 그림은 단일 변수의 분포를 보여줍니다(평활 히스토그램이라고 생각). 범주형 변수의 값에 따라 다른 분포를 보려면 범주에 따라 분포에 색을 다르게 지정할 수 있습니다. 예를 들어 TARGET = 1인지 0인지에 따라 색상이 지정된 이전_count_count의 커널 밀도 추정치를 표시할 수 있습니다. 결과 KDE는 대출금을 상환하지 않은 사람(TARGET == 1)과 상환한 사람(TARGET == 0) 간의 변수 분포에서 유의한 차이를 보여줍니다. 이는 변수가 기계 학습 모델과 '관련성'이 있는지 여부를 나타내는 지표가 될 수 있습니다.

In [ ]:
def kde_target(var_name, df):
  '''
  변수의 TARGET과의 상관 관계를 계산하고, 
  TARGET 값이 0인 경우와 1인 경우 각각의 변수 분포를 비교하여 시각화
  '''
  #상관관계 계산 
  corr = df['TARGET'].corr(df[var_name])
  
  #target이 1인 경우와 0인 경우의 각각의 중앙값
  avg_repaid = df.loc[df['TARGET']==0, var_name].median()
  avg_not_repaid = df.loc[df['TARGET']==1, var_name].median()

  #kde plot 생성성
  plt.figure(figsize=(12,6))
  sns.kdeplot(df.loc[df['TARGET']==0, var_name], label = 'TARGET == 0')
  sns.kdeplot(df.loc[df['TARGET']==1, var_name], label = 'TARGET == 1')

  plt.xlabel(var_name)
  plt.ylabel('Density')
  plt.title('%s Distribution' %var_name)
  plt.legend()

  print('The correlation between %s and the TARGET is %0.4f' %(var_name, corr))
  print('Median value for loan that was not repaid = %0.4f' % avg_not_repaid)
  print('Median value for loan that was repaid     = %0.4f' %avg_repaid)

랜덤 포레스트 및 그라데이션 부스팅 머신에 따르면 가장 중요한 변수 중 하나인 EXT_SOURCE_3 변수를 사용하여 이 함수를 테스트할 수 있습니다.

In [ ]:
kde_target('EXT_SOURCE_3', train)

이제 우리가 방금 만든 새로운 변수, 다른 기관의 이전 대출 수를 보겠습니다.

In [ ]:
kde_target('previous_loan_counts', train)

- 이로 인해 이 변수가 중요한지 여부를 판단하기가 어렵습니다. 상관 계수는 매우 약하며 분포에서 거의 눈에 띄는 차이가 없습니다.

- 다음으로 bureau 데이터 프레임에서 몇 가지 변수를 더 만들어 보겠습니다. bureau 데이터 프레임에 있는 모든 숫자 열의 평균, 최소값 및 최대값을 사용합니다.

## Aggregating Numeric Columns

 - bureau 데이터 프레임의 숫자 정보를 설명하기 위해 모든 숫자 열에 대한 통계를 계산할 수 있습니다. 
 
 - 이를 위해 클라이언트 ID별로 그룹화하고 그룹화된 데이터 프레임을 집계한 후 결과를 교육 데이터에 다시 병합합니다. 
 
 - agg 함수는 작업이 유효한 것으로 간주되는 숫자 열의 값만 계산합니다. 우리는 'mean', 'max', 'min', 'sum'을 계속 사용할 것이지만, 여기에는 어떤 기능도 전달될 수 있습니다. 우리는 우리 자신의 기능을 작성하여 agg 호출에 사용할 수도 있습니다.

In [ ]:
# 'SK_ID_BUREAU' 열을 삭제하고 'SK_ID_CURR'을 기준으로 그룹화하고,
# count, mean, max, sum  계산하여 bureau_agg 생성성

bureau_agg = bureau.drop(columns = ['SK_ID_BUREAU']).groupby('SK_ID_CURR', as_index = False
                                                             ).agg(['count', 'mean', 'max', 'sum']).reset_index()
bureau_agg.head()

- 각 열에 대해 새 이름을 만들어야 합니다. 다음 코드는 이름에 통계를 추가하여 새 이름을 만듭니다. 

- 여기서 우리는 데이터 프레임에 다단계 인덱스가 있다는 사실을 다루어야 합니다. 저는 이런 것들이 혼란스럽고 다루기가 어렵다고 생각하기 때문에 가능한 한 빨리 단일 수준의 색인으로 줄이려고 노력합니다.

In [ ]:
#bureau_agg 데이터프레임에서 각 변수 별로 count, mean, max, sum을 계산한 결과를 가지고 
#새로운 변수명을 생성하고 이를 columns 리스트에 추가

columns = ['SK_ID_CURR']

for var in bureau_agg.columns.levels[0]:
  if var != 'SK_ID_CURR':
    #각 변수마다 count, mean, max, sum을 계산한 결과에 대해서, 
    #새로운 변수명을 생성하고 columns 리스트에 추가
    for stat in bureau_agg.columns.levels[1][:-1]:
      columns.append('bureau_%s_%s' %(var, stat))

In [ ]:
bureau_agg.columns = columns
bureau_agg.head()

- 이제 이전과 같이 교육 데이터와 병합합니다

In [ ]:
train = train.merge(bureau_agg, on='SK_ID_CURR', how='left')
train.head()

### Correlations of Aggregated Values with Target

- 우리는 모든 새로운 값과 목표값의 상관관계를 계산할 수 있습니다. 다시, 우리는 이것들을 모델링에 중요할 수 있는 변수의 근사치로 사용할 수 있습니다.

In [ ]:
new_corrs = []

for col in columns:
  corr = train['TARGET'].corr(train[col])
  new_corrs.append((col, corr))

- 아래 코드에서는 정렬된 Python 함수를 사용하여 크기(절대값)에 따라 상관관계를 정렬합니다. 우리는 또한 알아두면 좋은 또 다른 중요한 파이썬 작업인 익명 람다 함수를 사용합니다.

In [ ]:
new_corrs = sorted(new_corrs, key=lambda x:abs(x[1]), reverse = True)
new_corrs[:15]

- 새 변수 중 대상과 유의한 상관 관계가 있는 변수가 없습니다. 상관관계가 가장 높은 변수인 bureau_의 KDE 그림을 볼 수 있습니다DAYS_Credit_mean(대상이 절대 크기 상관 관계)입니다.

In [ ]:
kde_target('bureau_DAYS_CREDIT_mean', train)

- 이 변수는 "현재 대출 신청 전에 고객이 Credit Bureau credit을 신청한 일 수"를 나타냅니다. 

- 내 해석은 이것이 홈 크레딧에서 대출을 신청하기 전에 이전 대출을 가입한 날짜의 수입니다. 따라서 더 큰 음수는 대출이 현재 대출 신청보다 더 이전에 신청되었음을 나타냅니다. 

- 이 변수의 평균과 target 간에는 매우 약한 양의 관계가 있습니다. 이는 이전에 대출을 신청한 고객이 Home Credit 대출을 상환할 가능성이 높다는 것을 나타냅니다. 그러나 상관관계가 이렇게 약하면 신호(signal)일 가능성과 노이즈(noise)일 가능성이 동등하게 높습니다.

#### The Multiple Comparisons Problem

- 많은 변수가 있을 때, 우리는 그들 중 일부가 순전히 우연에 의해 상관 관계를 맺을 것으로 예상합니다. 

- 이는 다중 비교로 알려진 문제입니다. 우리는 수백 개의 기능을 만들 수 있으며, 일부 기능은 단순히 데이터의 무작위 노이즈 때문에 대상과 연관되어 있는 것으로 밝혀질 것입니다. 

- 그런 다음, 우리 모델이 훈련할 때, 훈련 세트의 대상과 관계가 있다고 생각하기 때문에 이러한 변수에 지나치게 적합할 수 있지만, 이것이 반드시 테스트 세트로 일반화되는 것은 아닙니다. 기능을 만들 때 고려해야 할 많은 고려 사항이 있습니다!

##Function for Numeric Aggregations

앞의 모든 작업을 함수로 캡슐화해 보겠습니다. 이를 통해 모든 데이터 프레임에서 숫자 열의 집계 통계를 계산할 수 있습니다. 다른 데이터 프레임에도 동일한 작업을 적용하고자 할 때 이 기능을 다시 사용합니다.


In [ ]:
def agg_numeric(df, group_var, df_name):
  '''
  주어진 데이터프레임에서 수치형 열에 대한 집계 통계량을 계산하고, 
  지정된 그룹 변수(group_var)로 그룹화하여 결과를 반환
  '''
  for col in df:
    #group_var를 제외한 모든 'SK_ID'로 시작하는 열을 삭제
    if col != group_var and 'SK_ID' in col:
      df = df.drop(columns = col)
  
  group_ids = df[group_var]
  #number데이터만 선택
  numeric_df = df.select_dtypes('number')
  numeric_df[group_var] = group_ids

  #수치형 변수의 개수(count), 평균(mean), 최댓값(max), 최솟값(min), 합(sum)을 계산
  agg = numeric_df.groupby(group_var).agg(['count', 'mean', 'max', 'min', 'sum']).reset_index()

  columns = [group_var]

  for var in agg.columns.levels[0]:
    if var != group_var:
      for stat in agg.columns.levels[1][:-1]:
        columns.append('%s_%s_%s' %(df_name, var, stat))
    
  agg.columns = columns
  return agg

In [ ]:
bureau_agg_new = agg_numeric(bureau.drop(columns = ['SK_ID_BUREAU']), group_var = 'SK_ID_CURR', df_name = 'bureau')
bureau_agg_new.head()

기능이 의도한 대로 작동하는지 확인하려면 수작업으로 구축한 집계 데이터 프레임과 비교해야 합니다.

In [ ]:
bureau_agg.head()

우리가 그 값들을 조사해 보면, 우리는 그것들이 동등하다는 것을 알게 됩니다. 우리는 이 기능을 다른 데이터 프레임에 대한 숫자 통계를 계산하는 데 재사용할 수 있습니다. 기능을 사용하면 일관된 결과를 얻을 수 있고 앞으로 해야 할 일의 양이 줄어듭니다!



####Correlation Function

다음으로 넘어가기 전에 대상과의 상관관계를 계산하는 코드를 함수로 만들 수도 있습니다.

In [ ]:
def target_corrs(df):
  corrs = []
  for col in df.columns:
    print(col)
    if col != 'TARGET':
      corr = df['TARGET'].corr(df[col])
      corrs.append((col,corr))
  corrs = sorted(corrs, key=lambda x: abs(x[1]), reverse = True)

  return corrs

### Categorical Variables

이제 숫자 열에서 범주형 열로 이동합니다. 이러한 변수는 이산 문자열 변수이므로 숫자 변수에만 사용되는 평균 및 최대값과 같은 통계량을 계산할 수 없습니다. 대신 각 범주 변수 내에서 각 범주의 값 카운트를 계산합니다. 

In [ ]:
categorical = pd.get_dummies(bureau.select_dtypes('object'))
categorical['SK_ID_CURR'] = bureau['SK_ID_CURR']
categorical.head()

In [ ]:
categorical_grouped = categorical.groupby('SK_ID_CURR').agg(['sum', 'mean'])
categorical_grouped.head()

- 합 열은 연결된 클라이언트에 대한 해당 범주의 개수를 나타내고 평균은 정규화된 개수를 나타냅니다. 원핫 인코딩은 이러한 수치를 계산하는 과정을 매우 쉽게 만듭니다!

- 이전과 유사한 기능을 사용하여 열 이름을 바꿀 수 있습니다. 다시 말씀드리지만, 우리는 컬럼에 대한 다단계 인덱스를 처리해야 합니다. 범주 값(원핫 인코딩에서)이 추가된 범주 변수의 이름인 첫 번째 수준(수준 0)을 통해 반복합니다. 그런 다음 각 클라이언트에 대해 계산한 통계를 반복합니다. stat에 0 수준 이름이 추가된 열 이름을 변경합니다. 예를 들어 CREDIT_ACTIVE_ACTIVE가 레벨 0이고 합계가 레벨 1인 열은 CREDIT_ACTIVE_ACTIVE_count가 됩니다.

In [ ]:
categorical_grouped.columns.levels[0][:10]

In [ ]:
categorical_grouped.columns.levels[1]

In [ ]:
group_var = 'SK_ID_CURR'
columns =[]

for var in categorical_grouped.columns.levels[0]:
  if var != group_var:
    for stat in ['count', 'count_norm']:
      columns.append('%s_%s' %(var, stat))

categorical_grouped.columns = columns

categorical_grouped.head()

- 합 열에는 카운트가 기록되고 평균 열에는 정규화된 카운트가 기록됩니다.
- 이 데이터 프레임을 교육 데이터에 병합할 수 있습니다.

In [ ]:
train = train.merge(categorical_grouped, left_on = 'SK_ID_CURR', right_index = True, how='left')
train.head()

In [ ]:
train.shape

In [ ]:
train.iloc[:10, 123:]

#### Function to Handle Categorical Variables

코드를 더 효율적으로 만들기 위해 이제 범주형 변수를 처리하는 함수를 작성할 수 있습니다. 이것은 데이터 프레임과 그룹화 변수를 수용한다는 점에서 agg_numeric 함수와 동일한 형식을 취할 것입니다. 그런 다음 데이터 프레임의 모든 범주형 변수에 대한 각 범주의 카운트와 정규화된 카운트를 계산합니다.

In [ ]:
def count_categorical(df, group_var, df_name):
  categorical = pd.get_dummies(df.select_dtypes('object'))
  categorical[group_var] = df[group_var]

  categorical = categorical.groupby(group_var).agg(['sum', 'mean'])
  column_names = []

  for var in categorical.columns.levels[0]:
    for stat in ['count', 'count_norm']:
      column_names.append('%s_%s_%s' %(df_name, var, stat))

  categorical.columns = column_names
  return categorical

In [ ]:
bureau_counts = count_categorical(bureau, group_var ='SK_ID_CURR', df_name = 'bureau')
bureau_counts.head()

### Applying Operations to another dataframe

이제 bureau balance 데이터 프레임으로 넘어갑니다. 이 데이터 프레임에는 각 고객의 다른 금융 기관에 대한 이전 대출에 대한 월별 정보가 있습니다. 이 데이터 프레임을 SK로 그룹화하는 대신_클라이언트 ID인 ID_CURR, 먼저 데이터 프레임을 SK_별로 그룹화합니다이전 대출의 ID인 ID_Bureau입니다. 이렇게 하면 각 대출에 대해 데이터 프레임 행이 하나씩 제공됩니다. 그런 다음 SK_로 그룹화할 수 있습니다ID_CURR 및 각 클라이언트의 대출에 걸친 집계를 계산합니다. 최종 결과는 각 클라이언트에 대해 하나의 행이 있는 데이터 프레임이 되고, 각 클라이언트의 대출에 대해 통계가 계산됩니다.

In [ ]:
bureau_balance = pd.read_csv('bureau_balance.csv')
bureau_balance.head()

먼저, 각 대출에 대한 각 상태의 가치 카운트를 계산할 수 있습니다. 다행히도, 우리는 이미 이것을 해주는 기능을 가지고 있습니다!

In [ ]:
bureau_balance_counts = count_categorical(bureau_balance, group_var = 'SK_ID_BUREAU', df_name = 'bureau_balance')
bureau_balance_counts.head()

이제 하나의 숫자 열을 처리할 수 있습니다. MOONS_BALANCE 열에는 "적용 날짜를 기준으로 잔액이 있는 달"이 있습니다 이는 숫자 변수만큼 중요하지 않을 수 있으며, 향후 작업에서는 이를 시간 변수로 고려할 수 있습니다. 지금은 이전과 동일한 집계 통계만 계산하면 됩니다.

In [ ]:
bureau_balance_agg = agg_numeric(bureau_balance, group_var = 'SK_ID_BUREAU', df_name = 'bureau_balance')
bureau_balance_agg.head()

위의 데이터 프레임에는 각 대출에 대해 계산이 수행됩니다. 이제 각 클라이언트에 대해 이를 집계해야 합니다. 먼저 데이터 프레임을 병합한 다음 모든 변수가 숫자이므로 통계를 다시 집계하면 됩니다. 이번에는 SK_ID_CURR를 기준으로 그룹화하면 됩니다

In [ ]:
bureau_by_loan = bureau_balance_agg.merge(bureau_balance_counts, right_index = True, left_on = 'SK_ID_BUREAU', how = 'outer')

bureau_by_loan = bureau_by_loan.merge(bureau[['SK_ID_BUREAU', 'SK_ID_CURR']], on = 'SK_ID_BUREAU', how = 'left')
bureau_by_loan.head()

In [ ]:
bureau_balance_by_client = agg_numeric(bureau_by_loan.drop(columns = ['SK_ID_BUREAU']), group_var = 'SK_ID_CURR', df_name = 'client')
bureau_balance_by_client.head()

- 요약하자면 bureau_balance 데이터 프레임에 대해 다음과 같이 설명합니다:
  - 각 대출별로 계산된 숫자 통계 그룹화
  - 대출별 범주형 변수 그룹화의 만든 값 수
  - 대출에 대한 통계 및 값 개수 병합
  - 클라이언트 ID를 기준으로 결과 데이터 프레임 그룹화에 대해 계산된 숫자 통계

- 최종 결과 데이터 프레임에는 각 클라이언트에 대해 하나의 행이 있으며, 월별 잔액 정보로 모든 대출에 대해 통계가 계산됩니다.

- 이러한 변수 중 일부는 약간 혼란스러우므로 몇 가지 설명해 보겠습니다:
  - client_bureau_balance_MONGS_BALANCE_mean_mean: 각 대출에 대해 MONGS_BALANCE의 평균값을 계산합니다. 그런 다음 각 고객에 대해 모든 대출에 대한 이 값의 평균을 계산합니다.
  - client_bureau_balance_STATUS_X_count_norm_sum: 각 대출에 대해 발생 횟수 STATUS == X를 대출에 대한 총 STATUS 값으로 나눈 값을 계산합니다. 그런 다음 각 클라이언트에 대해 각 대출의 값을 합산합니다.

- 우리는 모든 변수가 하나의 데이터 프레임에 통합될 때까지 상관 관계를 계산하는 것을 보류할 것입니다.

#Putting the Functions Together

우리는 이제 다른 기관의 이전 대출금과 이 대출금에 대한 월별 지급 정보를 가져와서 주요 교육 데이터 프레임에 넣을 수 있는 모든 자료를 준비했습니다. 모든 변수를 재설정하고 처음부터 다시 설정하기 위해 만든 기능을 사용해 보겠습니다. 이것은 반복 가능한 워크플로우를 위해 기능을 사용하는 것의 이점을 보여줍니다!


In [ ]:
import gc
gc.enable()
del train, bureau, bureau_balance, bureau_agg, bureau_agg_new, bureau_balance_agg, bureau_balance_counts, bureau_by_loan, bureau_balance_by_client, bureau_counts
gc.collect()

In [ ]:
train = pd.read_csv('application_train.csv')
bureau = pd.read_csv('bureau.csv')
bureau_balance = pd.read_csv('bureau_balance.csv')

#### Count of Bureau DataFrame

In [ ]:
bureau_counts = count_categorical(bureau, group_var = 'SK_ID_CURR', df_name = 'bureau')
bureau_counts.head(0)

####Aggregated Stats of Bureau DataFrame

In [ ]:
bureau_agg = agg_numeric(bureau.drop(columns = ['SK_ID_BUREAU']), group_var = 'SK_ID_CURR', df_name = 'bureau')
bureau_agg.head()

####Value counts of Bureau Balance dataframe by loan


In [ ]:
bureau_balance_counts = count_categorical(bureau_balance, group_var = 'SK_ID_BUREAU', df_name = 'bureau_balance')
bureau_balance_counts.head()

#### Aggregated stats of Bureau Balance dataframe by loan

In [ ]:
bureau_balance_agg = agg_numeric(bureau_balance, group_var = 'SK_ID_BUREAU', df_name = 'bureau_balance')
bureau_balance_agg.head()

#### Aggregated Stats of Bureau Balance by Client

In [ ]:
bureau_by_loan = bureau_balance_agg.merge(
    bureau_balance_counts, right_index=True, left_on = 'SK_ID_BUREAU', how = 'left')

bureau_by_loan = bureau[['SK_ID_BUREAU', 'SK_ID_CURR']].merge(bureau_by_loan, on = 'SK_ID_BUREAU', how ='left')

bureau_balance_by_client = agg_numeric(bureau_by_loan.drop(columns = ['SK_ID_BUREAU']), group_var = 'SK_ID_CURR', df_name = 'client')

### Insert Computed Features into Training Data

In [ ]:
original_features = list(train.columns)
print('Original Number of Features: ', len(original_features))

In [ ]:
train = train.merge(bureau_counts, on = 'SK_ID_CURR', how = 'left')
train = train.merge(bureau_agg, on = 'SK_ID_CURR', how = 'left')
train = train.merge(bureau_balance_by_client, on = 'SK_ID_CURR', how = 'left')

In [ ]:
new_features = list(train.columns)
print('Number of features using previous loans from other institutions data: ', len(new_features))

#Feature Engineering Outcomes
- 그 모든 작업이 끝난 후, 이제 우리는 우리가 만든 변수들을 살펴보려고 합니다. 결측값의 백분율, 변수와 대상의 상관 관계 및 변수와 다른 변수의 상관 관계를 살펴볼 수 있습니다. 변수 간의 상관 관계는 공선형 변수, 즉 서로 상관 관계가 높은 변수가 있는지 여부를 보여줄 수 있습니다. 종종 두 변수를 모두 갖는 것은 중복되기 때문에 공선형 변수 쌍에서 하나를 제거하려고 합니다. 또한 결측값의 백분율을 사용하여 존재하지 않는 값의 대부분이 포함된 형상을 제거할 수 있습니다. 특징의 수를 줄이는 것은 모델이 훈련 중에 학습하는 데 도움이 되고 테스트 데이터에 더 잘 일반화될 수 있기 때문에 앞으로 특징 선택이 중요할 것입니다. 차원의 저주는 너무 많은 기능(차원이 너무 높음)으로 인해 발생하는 문제에 지정된 이름입니다. 변수의 수가 증가함에 따라 이러한 변수와 목표값 간의 관계를 학습하는 데 필요한 데이터 점의 수가 기하급수적으로 증가합니다.

- 기능 선택은 모델이 테스트 세트에 대해 더 잘 학습하고 일반화하는 데 도움이 되는 변수를 제거하는 프로세스입니다. 목적은 유용한 변수를 보존하면서 불필요한/중복 변수를 제거하는 것입니다. 이 프로세스에 사용할 수 있는 도구는 여러 가지가 있지만, 이 노트북에서는 결측값의 비율이 높은 열과 서로 상관 관계가 높은 변수를 제거하는 작업을 수행합니다. 나중에 그라데이션 부스팅 머신 또는 랜덤 포레스트와 같은 모델에서 반환된 기능 중요도를 사용하여 기능 선택을 수행하는 방법을 살펴볼 수 있습니다.


###Missing Values
중요한 고려 사항은 데이터 프레임의 결측값입니다. 결측값이 너무 많은 열은 삭제해야 할 수 있습니다.

In [ ]:
def missing_values_table(df):
  mis_val = df.isnull().sum()
  mis_val_percent = 100*df.isnull().sum()/len(df)
  mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
  mis_val_table_ren_columns = mis_val_table.rename(
      columns = {0 : 'Missing Values', 1 : '% of Total Values'}
  )
  mis_val_table_ren_columns = mis_val_table_ren_columns[
      mis_val_table_ren_columns.iloc[:,1] != 0].sort_values('% of Total Values', ascending=False).round(1)

  print('Your selected dataframe has ' + str(df.shape[1]) + ' columns.\n'
  'There are ' + str(mis_val_table_ren_columns.shape[0]) + 'columns that hace missing values.')

  return mis_val_table_ren_columns

In [ ]:
missing_train = missing_values_table(train)
missing_train.head(10)

결측값의 비율이 높은 열이 여러 개 있습니다. 결측값을 제거하기 위한 정확한 임계값은 없으며 문제에 따라 최적의 조치가 달라집니다. 여기에서는 피쳐 수를 줄이기 위해 교육 또는 테스트 데이터에서 결측값이 90%보다 큰 열을 제거합니다.

In [ ]:
missing_train_vars = list(missing_train.index[missing_train['% of Total Values']> 90])
len(missing_train_vars)

결측값을 제거하기 전에 검정 데이터에서 결측값 백분율을 찾습니다. 그런 다음 교육 또는 테스트 데이터에서 결측값이 90% 이상인 열을 제거합니다. 이제 테스트 데이터를 읽고 동일한 작업을 수행한 후 테스트 데이터의 결측값을 살펴보겠습니다. 이미 모든 카운트 및 집계 통계를 계산했으므로 테스트 데이터를 적절한 데이터와 병합하기만 하면 됩니다.

####Calculate Information for Testing Data

In [ ]:
test = pd.read_csv('application_test.csv')
test = test.merge(bureau_counts, on = 'SK_ID_CURR', how='left')
test = test.merge(bureau_agg, on = 'SK_ID_CURR', how = 'left')
test = test.merge(bureau_balance_by_client, on= 'SK_ID_CURR', how = 'left')

In [ ]:
print('Shape of Testing Data: ', test.shape)

우리는 테스트 및 교육 데이터 프레임을 정렬해야 합니다. 즉, 열이 정확히 동일하도록 열을 일치시키는 것입니다. 이것은 여기서 문제가 되지 않지만, 원핫 인코딩 변수를 사용할 때 데이터 프레임을 정렬하여 동일한 열이 있는지 확인해야 합니다.

In [ ]:
train_labels = train['TARGET']
train, test = train.align(test, join = 'inner', axis = 1)
train['TARGET']=train_labels

In [ ]:
print('Training Data Shape: ', train.shape)
print('Testing Data Shape: ', test.shape)

- 이제 데이터 프레임에 동일한 열이 있습니다(교육 데이터의 대상 열 제외). 이는 훈련 및 테스트 데이터 프레임 모두에서 동일한 열을 확인해야 하는 기계 학습 모델에서 사용할 수 있음을 의미합니다.

- 이제 검정 데이터에서 결측값의 백분율을 확인하여 삭제해야 할 열을 알아보겠습니다.

In [ ]:
missing_test = missing_values_table(test)
missing_test.head(10)

In [ ]:
missing_test_vars = list(missing_test.index[missing_test['% of Total Values'] > 90])
len(missing_test_vars)

In [ ]:
missing_columns = list(set(missing_test_vars + missing_train_vars))
print('There are %d columns with more than 90%% missing in either the training or testing data.' % len(missing_columns))

In [ ]:
train = train.drop(columns = missing_columns)
test = test.drop(columns = missing_columns)

- 결측값이 90%를 초과하는 열이 없기 때문에 이 라운드에서 열을 제거했습니다. 차원을 줄이기 위해 다른 형상 선택 방법을 적용해야 할 수도 있습니다.

- 이 시점에서 교육 및 테스트 데이터를 모두 저장합니다. 누락된 열을 떨어뜨리는 데 다른 비율을 사용하여 결과를 비교해 볼 것을 권장합니다.

In [ ]:
train.to_csv('train_bureau_raw.csv', index = False)
test.to_csv('test_bureau_raw.csv', index = False)

### Correlations
먼저 변수와 대상의 상관관계를 살펴보겠습니다. 우리는 우리가 만든 모든 변수에서 (응용 프로그램에서) 이미 교육 데이터에 있는 변수보다 더 큰 상관관계를 가지고 있다는 것을 알 수 있습니다.


In [ ]:
corrs = train.corr()

In [ ]:
corrs = corrs.sort_values('TARGET', ascending = False)
pd.DataFrame(corrs['TARGET'].head(10))

In [ ]:
pd.DataFrame(corrs['TARGET'].dropna().tail(10))

- 대상과 가장 높은 상관 관계가 있는 변수(물론 1의 상관 관계가 있는 대상 제외)는 우리가 만든 변수입니다. 하지만 변수가 상관관계가 있다고 해서 유용한 것은 아니며, 수백 개의 새로운 변수를 생성하면 일부 변수는 단순히 무작위 노이즈 때문에 대상과 상관관계가 있다는 것을 기억해야 합니다.

- 상관 관계를 회의적으로 보면 새로 생성된 변수 중 몇 가지가 유용할 수 있습니다. 변수의 "유용성"을 평가하기 위해 모형에서 반환되는 기능 중요도를 살펴보겠습니다. 호기심을 위해 (그리고 함수를 이미 작성했기 때문에) 새로 생성된 변수 중 두 개의 kde 플롯을 만들 수 있습니다.

In [ ]:
kde_target(var_name='client_bureau_balance_MONTHS_BALANCE_count_mean', df=train)

이 변수는 각 클라이언트의 대출당 월 평균 레코드 수를 나타냅니다. 예를 들어, 한 고객이 월별 데이터에 3, 4, 5개의 레코드가 있는 3개의 이전 대출을 가지고 있는 경우, 이 변수의 값은 4가 됩니다. 분포를 보면 대출 1건당 월평균 기록이 많은 고객일수록 홈크레디트로 대출금을 상환할 가능성이 높았습니다. 이 값을 너무 많이 읽지는 않지만, 이전에 신용 기록이 더 많은 고객이 일반적으로 대출을 상환할 가능성이 더 높다는 것을 나타낼 수 있습니다.

In [ ]:
kde_target(var_name = 'bureau_CREDIT_ACTIVE_Active_count_norm', df=train)

이런 분포는 도처에 널려 있습니다. 이 변수는 Credit_ACTIVE 값이 Active인 이전 대출의 수를 클라이언트에 대한 이전 대출의 총 수로 나눈 값입니다. 여기서의 상관관계는 너무 약해서 우리가 어떤 결론을 도출해야 한다고 생각하지 않습니다!

#### Collinear Variables

- 변수와 대상의 상관 관계뿐만 아니라 각 변수와 다른 모든 변수의 상관 관계도 계산할 수 있습니다. 이를 통해 데이터에서 제거해야 할 매우 선형적인 변수가 있는지 확인할 수 있습니다.

- 다른 변수와 상관 관계가 0.8보다 큰 변수를 찾습니다.


In [ ]:
threshold = 0.8
above_threshold_vars={}
for col in corrs:
  above_threshold_vars[col] = list(corrs.index[corrs[col] > threshold])

이러한 고도로 상관된 변수 쌍 각각에 대해 변수 중 하나만 제거하려고 합니다. 다음 코드는 각 쌍 중 하나만 추가하여 제거할 변수 집합을 만듭니다.

In [ ]:
cols_to_remove = []
cols_seen = []
cols_to_remove_pair = []

for key, value in above_threshold_vars.items():
    cols_seen.append(key)
    for x in value:
        if x == key:
            next
        else:
            if x not in cols_seen:
                cols_to_remove.append(x)
                cols_to_remove_pair.append(key)

cols_to_remove = list(set(cols_to_remove))
print('Number of columns to remove: ', len(cols_to_remove))

교육 및 테스트 데이터 세트 모두에서 이러한 열을 제거할 수 있습니다. 이러한 변수를 제거한 후의 성능과 이러한 변수(이전에 저장한 원시 csv 파일)를 유지하는 성능을 비교해야 합니다.

In [ ]:
train_corrs_removed = train.drop(columns = cols_to_remove)
test_corrs_removed = test.drop(columns = cols_to_remove)

print('Training Corrs Removed Shape: ', train_corrs_removed.shape)
print('Testing Corrs Removed Shape: ', test_corrs_removed.shape)

In [ ]:
train_corrs_removed.to_csv('train_bureau_corrs_removed.csv', index = False)
test_corrs_removed.to_csv('test_bureau_corrs_removed.csv', index=False)

# Modeling

- 이러한 새로운 데이터 세트의 성능을 실제로 테스트하기 위해 머신 러닝에 사용해 보겠습니다! 여기에서는 다른 노트북에서 개발한 기능을 사용하여 기능을 비교합니다(상관성이 높은 변수가 제거된 원시 버전). 우리는 이런 종류의 실험을 실행할 수 있으며, 경쟁사에 제출할 때 이 기능에 포함된 응용프로그램 데이터의 성능만 제어할 수 있습니다. 해당 성능을 이미 기록했으므로 제어 장치와 두 가지 테스트 조건을 나열할 수 있습니다:

- 모든 데이터 세트에 대해 아래에 표시된 모델(정확한 하이퍼 파라미터 포함)을 사용합니다.

  - control: 응용 프로그램 파일의 데이터만 표시됩니다.
  - test 1: 응용 프로그램 파일의 데이터와 Bureau_balance 파일에서 기록된 모든 데이터
  - test 2: 응용 프로그램 파일의 데이터와 bookief_balance 파일에서 모든 데이터가 기록되고 상관 관계가 높은 변수가 제거되었습니다.


In [ ]:
import lightgbm as lgb

from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder

import gc
import matplotlib.pyplot as plt

In [ ]:
def model(features, test_features, encoding='ohe', n_folds=5):
  #ids 추출
  train_ids = features['SK_ID_CURR']
  test_ids = test_features['SK_ID_CURR']
  #트레이닝의 라벨 추출
  labels = features['TARGET']
  
  #ids와 target 삭제
  features = features.drop(columns = ['SK_ID_CURR', 'TARGET'])
  test_features = test_features.drop(columns =['SK_ID_CURR'])

  #On hot Encoding
  if encoding == 'ohe':
    features = pd.get_dummies(features)
    test_features = pd.get_dummies(test_features)

    features, test_features = features.align(test_features, join = 'inner', axis=1)
    cat_indices = 'auto'

  elif encoding == 'le':
    label_encoder = LabelEncoder()
    cat_indices = []

    for i, col in enumerate(features):
      if features[col].dtype == 'object':
        features[col] = label_encoder.fit_transform(np.array(features[col].astype(str)).reshape((-1)))
        test_features[col] = label_encoder.transform(np.array(test_features[col].astype(str)).reshape((-1)))
        cat_indices.append(i)
  
  else:
    raise ValueError("Encoding must be either 'ohe' or 'le'")
  
  print('Training Data Shape: ', features.shape)
  print('Testing Data Shape: ', test_features.shape)
  
  feature_names = list(features.columns)
  features = np.array(features)
  test_features = np.array(test_features)

  k_fold = KFold(n_splits = n_folds, shuffle = False)

  feature_importance_values = np.zeros(len(feature_names))
  
  test_predictions = np.zeros(test_features.shape[0])

  out_of_fold = np.zeros(features.shape[0])

  valid_scores = []
  train_scores = []

  for train_indices, valid_indices in k_fold.split(features):
    train_features, train_labels = features[train_indices], labels[train_indices]
    valid_features, valid_labels = features[valid_indices], labels[valid_indices]
    model = lgb.LGBMClassifier(n_estimators=10000, objective = 'binary', 
                               class_weight = 'balanced', learning_rate = 0.05,
                               reg_alpha = 0.1, reg_lambda=0.1,
                               subsample = 0.8, n_jobs = -1, random_state = 50)
    
    model.fit(train_features, train_labels, eval_metric='auc',
                eval_set = [(valid_features, valid_labels),(train_features, train_labels)],
                eval_names = ['valid', 'train'], categorical_feature = cat_indices,
              early_stopping_rounds = 100, verbose = 200)
    best_iteration = model.best_iteration_
    feature_importance_values += model.feature_importances_ /k_fold.n_splits

    test_predictions += model.predict_proba(test_features, num_iteration = best_iteration)[:, 1] / k_fold.n_splits
    out_of_fold[valid_indices] = model.predict_proba(valid_features, num_iteration = best_iteration)[:, 1]

    valid_score = model.best_score_['valid']['auc']
    train_score = model.best_score_['train']['auc']

    valid_scores.append(valid_score)
    train_scores.append(train_score)

    gc.enable()
    del model, train_features, valid
    gc.collect()

    submission = pd.DataFrame({'SK_ID_CURR': test_ids, 'TARGET': test_predictions})
    feature_importances = pd.DataFrame({'feature': feature_names, 'importance': feature_importance_values})

    valid_auc = roc_auc_score(labels, out_of_fold)

    valid_scores.append(valid_auc)
    train_scores.append(np.mean(train_scores))

    fold_names = list(range(n_folds))
    fold_names.append('overall')

    metrics = pd.DataFrame({'fold': fold_names,
                            'train': train_scores,
                            'valid': valid_scores})
    
    return submission, feature_importances, metrics

In [ ]:
def plot_feature_importances(df):
  df = df.sort_values('importance',ascending = False).reset_index()
  df['importance_normalized'] = df['importance']/df['importance'].sum()

  plt.figure(figsize = (10,6))
  ax = plt.subplot()

  ax.barh(list(reversed(list(df.index[:15]))),
          df['importance_normalized'].head(15),
          align = 'center', edgecolor = 'k')
  
  ax.set_yticks(list(reversed(list(df.index[:15]))))
  ax.set_yticklabels(df['feature'].head(15))

  plt.xlabel('Normalized Importance') 
  plt.title('Feature Importances')
  plt.show()

  return df

####Control
모든 실험의 첫 번째 단계는 관리 수준을 설정하는 것입니다. 이를 위해 위에 정의된 기능(그라데이션 부스팅 머신 모델을 구현하는 기능)과 단일 메인 데이터 소스(애플리케이션)를 사용합니다.


In [ ]:
train_control = pd.read_csv('application_train.csv')
test_control = pd.read_csv('application_test.csv')

다행히도 함수를 작성하는 데 시간을 들인 후에는 함수를 사용하는 것이 간단합니다(이 노트에 중심 테마가 있는 경우에는 함수를 사용하여 작업을 단순하고 재현 가능하게 합니다!). 위의 함수는 경쟁사에 업로드할 수 있는 제출 데이터 프레임, 기능 중요도의 fi 데이터 프레임 및 검증 및 테스트 성능이 포함된 메트릭 데이터 프레임을 반환합니다.

In [ ]:
submission, fi, metrics = model(train_control, test_control)

In [ ]:
metrics

- 교육 점수가 유효성 검사 점수보다 높기 때문에 컨트롤이 약간 오버핏됩니다. 정규화를 검토할 때 이후 노트북에서 이 문제를 해결할 수 있습니다(이 모델에서는 이미 reg_lambda 및 reg_alpha와 조기 중지를 사용하여 일부 정규화를 수행합니다).

- flot_feature_importance라는 다른 함수를 사용하여 형상 중요도를 시각화할 수 있습니다. 형상 중요도는 형상을 선택할 때 유용할 수 있습니다.


In [ ]:
fi_sorted = plot_feature_importances(fi)

In [ ]:
submission.to_csv('contorl.csv', index=False)

컨트롤은 경기에 제출할 때 0.745점을 받습니다

##### Test One
첫 번째 테스트를 진행하겠습니다. 우리는 대부분의 작업을 수행하는 기능에 데이터를 전달하기만 하면 됩니다.

In [ ]:
submission_raw, fi_raw, metrics_raw = model(train,test)

In [ ]:
metrics_raw

이러한 수치를 바탕으로 설계된 기능이 제어 케이스보다 더 나은 성능을 발휘합니다. 그러나 이보다 더 나은 검증 성능이 테스트 데이터로 이전되는지 여부를 판단하기 전에 예측을 리더보드에 제출해야 합니다.

In [ ]:
fi_raw_sorted = plot_feature_importances(fi_raw)

기능 개선 사항을 검토해 보면, 우리가 구성한 기능 중 몇 가지가 가장 중요한 기능인 것 같습니다. 이 노트북에서 만든 가장 중요한 100가지 기능 중에서 가장 중요한 기능의 비율을 알아보겠습니다. 그러나 단순히 원래 기능과 비교하는 것이 아니라 일회성으로 인코딩된 원래 기능과 비교해야 합니다. 이것들은 이미 (원래 데이터에서) fi에 기록되어 있습니다.

In [ ]:
top_100 = list(fi_raw_sorted['feature'])[:100]
new_features = [x for x in top_100 if x not in list(fi['feature'])]
print('%% of Teop 100 Feautres created from the bureau data = %d.00' %len(new_features))


상위 100개 기능 중 절반 이상이 저희가 만들었습니다! 그것은 우리가 한 모든 노력이 가치가 있다는 확신을 줄 것입니다.

In [ ]:
submission_raw.to_csv('test_one.csv', index = False)

테스트 1은 경기에 제출할 때 0.759점을 받습니다.

####Test Two
쉬웠으니까 한 번 더 뛰자구요! 이전과 동일하지만 매우 공선형 변수가 제거되었습니다.

In [ ]:
submission_corrs, fi_corrs, metrics_corr = model(train_corrs_removed, test_corrs_removed)

In [ ]:
metrics_corr

이러한 결과는 관리보다는 낫지만 원시 피쳐보다는 약간 낮습니다.

In [ ]:

fi_corrs_sorted = plot_feature_importances(fi_corrs)

In [ ]:
submission_corrs.to_csv('test_two.csv', index=False)

테스트 2는 경기에 제출할 때 0.753점을 받습니다.

#Results

- 그 모든 작업 후에, 우리는 추가 정보를 포함하는 것이 성능을 향상시켰다고 말할 수 있습니다! 이 모델은 확실히 데이터에 최적화되지 않았지만 계산된 기능을 사용할 때 원래 데이터 세트보다 눈에 띄게 개선되었습니다. 성과를 공식적으로 요약해 보겠습니다:

- 실험 열차 AUC 검증 AUC 테스트 AUC
제어 0.8150.7600.745
검정 1 0.8370.7670.759
검정 2개 0.826 0.765 0.753
이러한 점수는 노트북의 실행에 따라 변경될 수 있습니다. 하지만 저는 일반적인 주문이 바뀌는 것을 보지 못했습니다.)

- 우리의 모든 노력은 원래 테스트 데이터보다 0.014 ROCAUC의 작은 개선으로 이어집니다. 고도로 공선형 변수를 제거하면 성능이 약간 저하되므로 피쳐 선택을 위한 다른 방법을 고려하려고 합니다. 또한, 우리가 만든 기능 중 일부는 모델에 의해 판단되는 가장 중요한 기능 중 하나라고 말할 수 있습니다.

- 이와 같은 경쟁에서 이 크기의 개선만으로도 리더보드에서 100개의 자리를 차지할 수 있습니다. 이 노트북과 같이 여러 가지 작은 개선을 통해 점차 더 나은 성능을 달성할 수 있습니다. 저는 다른 사람들이 이 결과를 사용하여 스스로 개선할 것을 권장하며, 다른 사람들을 돕기 위해 제가 취한 조치를 계속 문서화할 것입니다.

#Next Step
앞으로 이 노트북에서 개발한 기능을 다른 데이터 세트에서 사용할 수 있습니다. 모델에 사용할 다른 4개의 데이터 파일이 남아 있습니다! 다음 노트북에서는 이러한 다른 데이터 파일의 정보(홈 크레딧의 이전 대출에 대한 정보 포함)를 교육 데이터에 통합합니다. 그런 다음 동일한 모델을 구축하고 더 많은 실험을 실행하여 기능 엔지니어링의 효과를 확인할 수 있습니다. 이 경쟁에서 더 많은 일을 해야 하고, 더 많은 성과를 내야 합니다! 다음 공책에서 뵙겠습니다.